In [1]:
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#读取数据
data = pd.read_excel('./育龄妇女数.xls')
# 先训练总人口
train = data['灰色预测'].values[:18] #训练数据
test = data['灰色预测'].values[18:31] #测试数据
data

,时间,平均育龄妇女人数(人),灰色预测,神经网络预测
0,2003,350677.0,350667.000000,NaN
1,2004,351232.0,368235.100000,NaN
2,2005,467659.0,362428.400000,NaN
3,2006,328924.0,356713.300000,NaN
4,2007,324810.0,351088.300000,NaN
5,2008,319513.0,345552.100000,NaN
6,2009,317359.0,340103.100000,NaN
7,2010,308204.0,334740.000000,NaN
8,2011,325497.0,329461.600000,NaN
9,2012,317046.0,324466.300000,NaN


In [2]:
test

array([281044.8, 276613. , 272251.2, 267958.1, 263732.7, 259573.9,
       255480.7, 251452. , 267880. , 263709.7, 259604.4, 255562.9,
       251584.4])

In [3]:
def GM11(x,n):
    '''
    灰色预测
    x：序列，numpy对象
    n:需要往后预测的个数
    '''
    x1 = x.cumsum()#一次累加  
    z1 = (x1[:len(x1) - 1] + x1[1:])/2.0#紧邻均值  
    z1 = z1.reshape((len(z1),1))  
    B = np.append(-z1,np.ones_like(z1),axis=1)  
    Y = x[1:].reshape((len(x) - 1,1))
    
    #a为发展系数 b为灰色作用量
    [[a],[b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)#计算参数  
    result = (x[0]-b/a)*np.exp(-a*(n-1))-(x[0]-b/a)*np.exp(-a*(n-2))  
    S1_2 = x.var()#原序列方差
    e = list()#残差序列
    for index in range(1,x.shape[0]+1):
        predict = (x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2))
        e.append(x[index-1]-predict)
    S2_2 = np.array(e).var()#残差方差
    C = S2_2/S1_2#后验差比
    if C<=0.35:
        assess = '后验差比<=0.35，模型精度等级为好'
    elif C<=0.5:
        assess = '后验差比<=0.5，模型精度等级为合格'
    elif C<=0.65:
        assess = '后验差比<=0.65，模型精度等级为勉强'
    else:
        assess = '后验差比>0.65，模型精度等级为不合格'
        
    #预测数据
    predict = list()
    for index in range(x.shape[0]+1,x.shape[0]+n+1):
        predict.append((x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2)))
    predict = np.array(predict)
    return {
            'a':{'value':a,'desc':'发展系数'},
            'b':{'value':b,'desc':'灰色作用量'},
            'predict':{'value':result,'desc':'第%d个预测值'%n},
            'C':{'value':C,'desc':assess},
            'predict':{'value':predict,'desc':'往后预测%d个的序列'%(n)},
            }

In [4]:
#GM11动态建模
yPre = []
for i in range(test.shape[0]):
    #只预测1个数
    result = GM11(train,1)
    yPre.append(result['predict']['value'][0])
    #更新训练集
    train = train.tolist()[:-1]
    train.append(test[i])
    train = np.array(train).reshape(-1)
#计算MAE
MAE = mean_absolute_error(test,yPre)
#打印模型
print(result['C']['desc'])
print(result['a']['desc'],np.round(result['a']['value'],2))
print(result['b']['desc'],np.round(result['b']['value'],2))

后验差比<=0.35，模型精度等级为好
发展系数 0.02
灰色作用量 380615.46


In [21]:
# i = 21
# for i in range(2,41):
#     data = np.array([98705,100072,101654,103008,104357,105851,107507,109300,111026,112704,114333,115823,117171,118517,119850,121121,122389
#     ,123626,124761,125786,126743,127627,128453,129227,129988,130756,131448,132129,132802,133450,134091,134916,135922,136726,137646,138326
#     ,139232,140011,140541,141008,141212])
#     x =  data[:i] #输入数据 1994年之前
#     y =  data[i:i+1] #需要预测的数据 2001-2020
#     result = GM11(x,len(y))  
#     predict = result['predict']['value']
#     predict = np.round(predict,1)
#     print('第',1980+i,'的真实值:',y)
#     print('预测值:',predict)
#     print()

In [42]:
if __name__ == "__main__":
    data = np.array([350677,351232,467659,328924,324810,319513,317359,308204,325497,317046,308867,304733,365669,304291.7,299493.4,294770.7,290122.5
,285547.6
,281044.8
,276613
,272251.2
,267958.1
,263732.7
,259573.9
,255480.7
,251452
,267880
,263709.7
,259604.4
,255562.9
,251584.4
])
    x = data[:29] #输入数据
    y = data[29:] #需要预测的数据
    result = GM11(x,len(y))  
    predict = result['predict']['value']
    predict = np.round(predict,1)
    print('真实值:',y)
    print('预测值:',predict)

真实值: [255562.9 251584.4]
预测值: [242879.9 239416.7]


In [43]:
len(y)

2

In [44]:
print(result) # 2021-2035的相关数据

{'a': {'value': 0.014361542194514944, 'desc': '发展系数'}, 'b': {'value': 370754.19840219116, 'desc': '灰色作用量'}, 'predict': {'value': array([242879.88254561, 239416.68084759]), 'desc': '往后预测2个的序列'}, 'C': {'value': 0.35113028561849213, 'desc': '后验差比<=0.5，模型精度等级为合格'}}


In [45]:
print(list(result['predict']['value']))
print(len(list(result['predict']['value'])))

[242879.88254561275, 239416.68084759265]
2
